In [1]:
import pandas as pd
import numpy as np
import os
import torch
import torch.nn as nn
from sklearn.preprocessing import StandardScaler
from collections import Counter

In [2]:
FEATURE_COLS = ['grid', 'qual_pos', 'q_rel_gap', 'age_at_race', 'driver_form', 'team_form', 'driver_experience', 'circuit_avg_pos']
IMPORTANCE_MAP = {
    'team_form': 0.450637, 'driver_form': 0.189334, 'qual_pos': 0.094745,
    'circuit_avg_pos': 0.061325, 'driver_experience': 0.060547,
    'grid': 0.051296, 'age_at_race': 0.046638, 'q_rel_gap': 0.045478
}

In [3]:
class AttentionLayer(nn.Module):
    def __init__(self, hidden_dim):
        super().__init__()
        self.attn = nn.Linear(hidden_dim, 1)
    def forward(self, x):
        weights = torch.softmax(self.attn(x), dim=1)
        return torch.sum(weights * x, dim=1)

In [4]:
class F1UltimateHybridModel(nn.Module):
    def __init__(self, num_features, d_model=128, nhead=8):
        super().__init__()
        
        self.embedding = nn.Linear(num_features, d_model)
        encoder_layer = nn.TransformerEncoderLayer(d_model=d_model, nhead=nhead, batch_first=True)
        self.transformer = nn.TransformerEncoder(encoder_layer, num_layers=2)
        
        
        self.lstm = nn.LSTM(d_model, d_model, batch_first=True, bidirectional=True)
        
        
        self.attention = AttentionLayer(d_model * 2)
        
        
        self.fc = nn.Sequential(
            nn.Linear(d_model * 2, 64), 
            nn.GELU(),
            nn.LayerNorm(64),           
            nn.Linear(64, 1),           
            nn.Sigmoid()                
        )

    def forward(self, x):
        x = self.embedding(x)
        x = self.transformer(x)
        lstm_out, _ = self.lstm(x)
        context = self.attention(lstm_out)
        return self.fc(context)


In [5]:
def get_main_points(pos):
    pts = {1: 25, 2: 18, 3: 15, 4: 12, 5: 10, 6: 8, 7: 6, 8: 4, 9: 2, 10: 1}
    return pts.get(int(pos), 0)

In [6]:
def get_sprint_points(pos):
    pts = {1: 8, 2: 7, 3: 6, 4: 5, 5: 4, 6: 3, 7: 2, 8: 1}
    return pts.get(int(pos), 0)

In [7]:
def run_ultimate_simulation():
    data_dir = '../data'
    data_engineered_dir = '../data_engineered'
    model_path = '../models/f1_ultimate_hybrid.pth'
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    sim_df = pd.read_csv(os.path.join(data_engineered_dir, 'historical_sim_ready_v2.csv'), low_memory=False)
    results_df = pd.read_csv(os.path.join(data_dir, 'results.csv'))
    drivers_df = pd.read_csv(os.path.join(data_dir, 'drivers.csv'), encoding='ISO-8859-1')
    constructors_df = pd.read_csv(os.path.join(data_dir, 'constructors.csv'), encoding='ISO-8859-1')
    races_df = pd.read_csv(os.path.join(data_dir, 'races.csv'))

    scaler = StandardScaler()
    sim_df[FEATURE_COLS] = scaler.fit_transform(sim_df[FEATURE_COLS])
    for col in FEATURE_COLS:
        sim_df[col] = sim_df[col] * (1.0 + IMPORTANCE_MAP.get(col, 0.0))

    results_df = pd.merge(results_df, races_df[['raceId', 'year']], on='raceId', how='left')
    results_df['actual_main_pts'] = results_df['positionOrder'].apply(get_main_points)
    
    actual_driver_main = results_df.groupby(['year', 'driverId'])['actual_main_pts'].sum().reset_index()
    actual_driver_main['driver_name'] = actual_driver_main['driverId'].map(drivers_df.set_index('driverId')['forename'] + " " + drivers_df.set_index('driverId')['surname'])
    actual_driver_main['original_rank'] = actual_driver_main.groupby('year')['actual_main_pts'].rank(method='first', ascending=False)

    model = F1UltimateHybridModel(len(FEATURE_COLS)).to(device)
    if os.path.exists(model_path):
        model.load_state_dict(torch.load(model_path, map_location=device))
        model.eval()
        print(f"--- v3 Ultimate Hybrid load success (Device: {device}) ---")
    else:
        print("Model file not found.")
        return

    const_id_lookup = results_df.drop_duplicates(['raceId', 'driverId']).set_index(['raceId', 'driverId'])['constructorId'].to_dict()
    sim_df['constructorId'] = sim_df.apply(lambda x: const_id_lookup.get((int(x['raceId']), int(x['driverId'])), 0), axis=1)

    sprint_results = []
    SEQ_LEN = 3
    for driver_id, group in sim_df.groupby('driverId'):
        group = group.sort_values(['year', 'round'])
        if len(group) <= SEQ_LEN: continue
        f_vals = group[FEATURE_COLS].values
        for i in range(len(group) - SEQ_LEN):
            target_row = group.iloc[i + SEQ_LEN]
            if 1 <= int(target_row['round']) <= 6:
                seq_x = torch.FloatTensor(f_vals[i:i+SEQ_LEN]).unsqueeze(0).to(device)
                with torch.no_grad():
                    pred_score = model(seq_x).item()
                sprint_results.append({
                    'year': int(target_row['year']), 'round': int(target_row['round']),
                    'driverId': int(driver_id), 'constructorId': int(target_row['constructorId']),
                    'pred_score': pred_score
                })

    sprint_df = pd.DataFrame(sprint_results).drop_duplicates(['year', 'round', 'driverId'])
    sprint_df['pred_pos'] = sprint_df.groupby(['year', 'round'])['pred_score'].rank(method='first', ascending=True)
    sprint_df['sprint_pts'] = sprint_df['pred_pos'].apply(get_sprint_points)
    driver_sprint_sum = sprint_df.groupby(['year', 'driverId'])['sprint_pts'].sum().reset_index()

    final_comparison = pd.merge(actual_driver_main, driver_sprint_sum, on=['year', 'driverId'], how='left').fillna(0)
    final_comparison['total_pts'] = final_comparison['actual_main_pts'] + final_comparison['sprint_pts']
    final_comparison['new_rank'] = final_comparison.groupby('year')['total_pts'].rank(method='first', ascending=False)

    history_winners = []
    rank_flips = []

    for yr in sorted(final_comparison['year'].unique(), reverse=True):
        orig_winner = final_comparison[(final_comparison['year'] == yr) & (final_comparison['original_rank'] == 1)].iloc[0]
        new_winner = final_comparison[(final_comparison['year'] == yr) & (final_comparison['new_rank'] == 1)].iloc[0]
        history_winners.append(new_winner['driver_name'])

        print(f"\n" + "="*70)
        print(f" 🏁 {yr} SEASON ANALYSIS")
        print(f" - Actual Champion: {orig_winner['driver_name']}")
        
        if orig_winner['driverId'] != new_winner['driverId']:
            print(f" 🚨 [RANK FLIPPED!] -> New Champion: {new_winner['driver_name']}")
            rank_flips.append({'year': yr, 'from': orig_winner['driver_name'], 'to': new_winner['driver_name']})
        else:
            print(f" ✅ [CHAMPION RETAINED] : {new_winner['driver_name']}")
        print(f" - Points: {int(new_winner['total_pts'])} (Sprint contribution: +{int(new_winner['sprint_pts'])})")

    print("\n\n" + "#"*70)
    print(" 📊 FINAL SUMMARY: THE MOST DOMINANT IN AI-HYBRID ERA ")
    print("#"*70)
    
    print(f"\n[Years with Championship Flips]")
    for f in rank_flips:
        print(f"🚩 {f['year']}: {f['from']} ➔ {f['to']} (Title Changed by AI Sprint)")

    winner_counts = Counter(history_winners).most_common()
    print(f"\n[Most Driver Titles]")
    for name, count in winner_counts:
        print(f"- {name:<25} : {count} titles")
    print("#"*70)

In [8]:
if __name__ == "__main__":
    run_ultimate_simulation()

--- v3 Ultimate Hybrid load success (Device: cuda) ---

 🏁 2024 SEASON ANALYSIS
 - Actual Champion: Max Verstappen
 ✅ [CHAMPION RETAINED] : Max Verstappen
 - Points: 396 (Sprint contribution: +0)

 🏁 2023 SEASON ANALYSIS
 - Actual Champion: Max Verstappen
 ✅ [CHAMPION RETAINED] : Max Verstappen
 - Points: 521 (Sprint contribution: +0)

 🏁 2022 SEASON ANALYSIS
 - Actual Champion: Max Verstappen
 ✅ [CHAMPION RETAINED] : Max Verstappen
 - Points: 428 (Sprint contribution: +0)

 🏁 2021 SEASON ANALYSIS
 - Actual Champion: Max Verstappen
 ✅ [CHAMPION RETAINED] : Max Verstappen
 - Points: 396 (Sprint contribution: +0)

 🏁 2020 SEASON ANALYSIS
 - Actual Champion: Lewis Hamilton
 ✅ [CHAMPION RETAINED] : Lewis Hamilton
 - Points: 384 (Sprint contribution: +43)

 🏁 2019 SEASON ANALYSIS
 - Actual Champion: Lewis Hamilton
 ✅ [CHAMPION RETAINED] : Lewis Hamilton
 - Points: 453 (Sprint contribution: +46)

 🏁 2018 SEASON ANALYSIS
 - Actual Champion: Lewis Hamilton
 ✅ [CHAMPION RETAINED] : Lewis Hamilt